In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from src.plotting_tools.SysHist import make_hist, make_sys_hist, SysHist
from src.data_tools.get_data import get_data, delta_r_cut
from src.plotting_tools.Bins import Bins, bins, binning_type
from src.plotting_tools.cms_format import cms_format_fig, cms_style
from src.plotting_tools.colors import color_fader
from src.data_tools.StackPlotter import get_stack_plotter
from src.plotting_tools.latexAssets import HTLT, RelMET
from src.plotting_tools.utils import ratio_plot_template


In [ ]:
cms_style()

In [ ]:
era = 'Run 2'
fstring_era = era.replace(' ', '_')
from src.assets.output_dir import output_dir
output_dir

In [ ]:
sp6 = get_stack_plotter(output_dir, '2016', bins='none')
sp7 = get_stack_plotter(output_dir, '2017', bins='none')
sp8 = get_stack_plotter(output_dir, '2018', bins='none')

In [ ]:
sp = sp6
sp.plot_df = pd.concat([sp6.plot_df , sp7.plot_df, sp8.plot_df])
#sp.plot_df = pd.concat([sp.plot_df , sp8.plot_df])
sp.plot_df.era = 2000
sp.era=2000

In [ ]:
# add missing weights
keys = []
for i, row in sp.plot_df.iterrows():
    keys += list(row.sys.keys())
unique = np.unique(keys)
unique = list(filter( lambda x: "Weight" in x, unique))

for i, row in sp.plot_df.iterrows():
    keys = list(row.sys.keys())
    # don't do for data
    if len(keys) == 0: continue
    for w in unique:
        if w not in keys: row.sys[w] = [row.sys[keys[0]][0]*0,row.sys[keys[0]][0]]
    

In [ ]:
reg = 'SR1'
feature = 'HTLT'
sp.rebin =0
sp.x_range = (-np.inf, np.inf)
fig, ax = plt.subplots(figsize=(12,12))
x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
x = sp.draw_signals(ax, feature, reg, make_density=0)

ax.legend(ncol=2)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e7)
cms_format_fig("Run 2", ax, "\emph{Simulation}")
ax.set_ylabel("Events")
ax.set_xlabel(HTLT + " [GeV]")
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))

In [ ]:
reg = 'SR2'
feature = 'HTLT'
sp.scale = 1
sp.rebin =0
sp.x_range = (-np.inf, np.inf)
fig, ax = plt.subplots(figsize=(12,12))
x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
x = sp.draw_signals(ax, feature, reg, make_density=0)

ax.legend(ncol=2)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e7)
cms_format_fig("Run 2", ax, "\emph{Simulation}")
ax.set_ylabel("Events")
ax.set_xlabel(HTLT + " [GeV]")
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
sp.scale = 1

In [ ]:
reg = 'SR1'
feature = 'RelMET'
#scale by binning
sp.scale = 1
sp.rebin =0
sp.x_range = (-np.inf, np.inf)
fig, ax = plt.subplots(figsize=(12,12))
x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
x = sp.draw_signals(ax, feature, reg, make_density=0)
""
ax.legend(ncol=2)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e7)
cms_format_fig("Run 2", ax, "\emph{Simulation}")
ax.set_ylabel("Events")
ax.set_xlabel(RelMET)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
sp.scale = 1

In [ ]:
reg = 'SR2'
feature = 'RelMET'
#scale by binning
sp.scale = 1
sp.rebin =0
sp.x_range = (-np.inf, np.inf)
fig, ax = plt.subplots(figsize=(12,12))
x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
x = sp.draw_signals(ax, feature, reg, make_density=0)


ax.legend(ncol=2)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e7)
cms_format_fig("Run 2", ax, "\emph{Simulation}")
ax.set_ylabel("Events")
ax.set_xlabel(RelMET)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
sp.scale = 1

In [ ]:
def plot_reg_feature_ratio(reg, feature, xlabel='$\delta_{R}$'):
    fig, ax, rax = ratio_plot_template(figsize=(10,10))
    sp.scale = 1
    bhist = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
    _ = sp.draw_data(ax, feature, reg, make_density=0)
        
    #ratio
    x = bhist.calc_bin_centers()
    rax.plot(x, np.full(len(x), .5) , color='black', linestyle=':')
    rax.plot(x, np.full(len(x), 1) , color='black')
    rax.plot(x, np.full(len(x), 1.5) , color='black', linestyle=':') 
    rax.set_ylim(0,2)
    rax.set_ylabel('Obs./MC')
    if 'SR' in reg: 
        'none'
    else:
        _dhist = sp.make_data_hist(feature, reg).calc_ratio(bhist.nominal)
        rax.errorbar(_dhist.calc_bin_centers(), _dhist.nominal, yerr=_dhist.std, color='black',
                    label='Observed', ls='', marker='o')
    bhist.calc_ratio(bhist.nominal+1e-10).draw(rax, errorbar=False)
    
    
    ax.legend(ncol=2)
    ax.set_yscale('log')
    ax.set_ylim(1e-2, 1e5)
    cms_format_fig("Run 2", ax, "\emph{Preliminary}")
    ax.set_ylabel("Events")
    ax.set_xlabel(xlabel)
    fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))

In [ ]:
reg = 'CR24'
feature = 'minGoodJetElDR'
plot_reg_feature_ratio(reg, feature, xlabel='$\Delta_{R}$ min(Electron, Jet)')

In [ ]:
reg = 'CR20'
feature = 'minGoodJetMuDR'
plot_reg_feature_ratio(reg, feature, xlabel='$\Delta_{R}$ min(Muon, Jet)')

In [ ]:
import pickle as pkl
firstHist=1
reg_dict = {}
for _era in ["2016", "2017", "2018"]:
    #if _era !="2016": continue
    outname="{}/abcd/abcd_dict_data_{}_ismc0.pkl".format(output_dir, _era)
    with open(outname,'rb') as f:
        abcd = pkl.load(f)
        for reg in abcd.keys():
            if firstHist==1:
                reg_dict[reg] = SysHist.from_dict(abcd[reg])
            else:
                reg_dict[reg] += SysHist.from_dict(abcd[reg])
        firstHist=0

In [ ]:
def draw_dilep(reg):
    
    fig, ax, rax = ratio_plot_template(figsize=(10,10))
    
    
    feature = 'DiLepMass'

    sp.rebin =reg_dict[reg].bin_edges
    sp.x_range = (120, 400)
    bhist = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=1)
    #sp.scale = 10/1600
    _ = sp.draw_data(ax, feature, reg, make_density=0)
    
    if 'SR' in reg: kwargs = {"label": "ABCD", "zorder": 4}
    else: kwargs = {"label": "Observed Fit", "color": "red", "zorder": 4}
    reg_dict[reg].make_density_hist().draw(ax,**kwargs)
    
    ax.legend(ncol=2)
    ax.set_yscale('log')
    ax.set_ylim(1e-2, 1e5)
    cms_format_fig(era, ax, "\emph{Preliminary}")
    ax.set_ylabel("Events per GeV")
    
    ax.set_xlabel('$m_{\ell\ell}$ [GeV]')
    
    #ratio
    x = bhist.calc_bin_centers()
    rax.plot(x, np.full(len(x), .5) , color='black', linestyle=':')
    rax.plot(x, np.full(len(x), 1) , color='black')
    rax.plot(x, np.full(len(x), 1.5) , color='black', linestyle=':') 
    rax.set_ylim(0,2)
    rax.set_ylabel('Obs./MC')
    if 'SR' in reg: 
        rax.set_ylabel('ABCD/MC')
    else:
        _dhist = sp.make_data_hist(feature, reg).calc_ratio(bhist.nominal)
        rax.errorbar(_dhist.calc_bin_centers(), _dhist.nominal, yerr=_dhist.std, color='black',
                    label='Observed', ls='', marker='o')
        
    reg_dict[reg].make_density_hist().calc_ratio(bhist.nominal).draw(rax, **kwargs)
    bhist.calc_ratio(bhist.nominal).draw(rax, zorder=1, errorbar=False)
    fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
    sp.scale = 1

In [ ]:
for reg in reg_dict.keys():
    print(reg)
    draw_dilep(reg)

In [ ]:
!ls -rt /eos/cms/store/group/phys_exotica/bffZprime/assets_june_23/data

In [ ]:
##
## Compare different dbs distributions
##
def compare_dbs(reg, feature):
    #scale by binning
    sp.scale = 1
    sp.rebin =0
    sp.x_range = (-np.inf, np.inf)
    fig, ax = plt.subplots(figsize=(12,12))
    #x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
    x = sp.draw_signals_compare_dbs(ax, feature, reg, make_density=1, 
                                    dbs_values = [0.04], mass_values = [125., 350.])
    x = sp.draw_signals_compare_dbs(ax, feature, reg, make_density=1, 
                                    dbs_values = [1.0], mass_values = [125., 350.],
                                   c1='#00da2b', c2='#e67e22')    
    
    ax.legend(ncol=2)
    ax.set_yscale('log')
    ax.set_ylim(1e-5, 1e0)
    cms_format_fig("Run 2", ax, "\emph{Simulation}")
    ax.set_ylabel("Events")
    ax.set_xlabel(RelMET)
    fig.savefig('{}/stack_plots/dbs_comp_{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
    sp.scale = 1

In [ ]:
reg = 'SR2'
feature = 'RelMET'
compare_dbs(reg, feature)

In [ ]:
reg = 'SR1'
feature = 'RelMET'
compare_dbs(reg, feature)

In [ ]:
reg = 'SR1'
feature = 'HTLT'
compare_dbs(reg, feature)

In [ ]:
reg = 'SR2'
feature = 'HTLT'
compare_dbs(reg, feature)

In [ ]:
break

In [ ]:
era = '2016'
output_dir = 'assets_nov_2022'
df, lumi = get_data(era, output_dir, stitch_dy=1, filterdeltar=False)

In [ ]:
from src.assets.regions import nJets, regions, region_and_label
from src.assets.file_groups import bck_dict, bck_list, bck_colors


In [ ]:
from src.plotting_tools.draw_stack_plot_hists import draw_bckground, draw_signals, draw_data, draw_stackplot

In [ ]:
from src.plotting_tools.latexAssets import HTLT, RelMET

In [ ]:
draw_inline = True
if draw_inline:
    plt.ion()
else:
    plt.ioff()

In [ ]:
#HTLT relmet plt
# select_level=.5 ensures that its pre bff seleciton cuts
reg = 'SR1'
feature = 'HTLT'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=1, draw_sig_sys=0, top=1e5,  
               bin_edges=np.linspace(-800,800,100),
               make_density=0,
             select_level=.5,
              xlabel=HTLT)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))

reg = 'SR1'
feature = 'RelMET'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=1, draw_sig_sys=0, top=1e5, 
               bin_edges=np.linspace(0, 2,100),
               make_density=0,
              select_level=.5,
              xlabel= RelMET)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))



In [ ]:
#HTLT relmet plt
# select_level=.5 ensures that its pre bff seleciton cuts
reg = 'SR2'
feature = 'HTLT'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=1, draw_sig_sys=0, top=1e5, 
            make_density=0,
            bin_edges=np.linspace(-800,800,100),
            select_level=.5,
              xlabel=HTLT)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))

reg = 'SR2'
feature = 'RelMET'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=1, draw_sig_sys=0, top=1e5, 
            make_density=0,
            bin_edges=np.linspace(0, 2,100),
            select_level=.5,
              xlabel= RelMET)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))



In [ ]:
# delta r between jet and electron
reg = 'CR14'
feature = 'minGoodJetElDR'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=1, draw_sig_sys=1, top=1e5,  
               bin_edges=np.linspace(0,5,100),
               make_density=0,
             select_level=.5,
              xlabel='$\delta_{R}$')
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))


In [ ]:
df = delta_r_cut(df, 0.4)
# delta r between jet and electron
reg = 'CR14'
feature = 'minGoodJetElDR'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=1, draw_sig_sys=1, top=1e5,  
               bin_edges=np.linspace(0,5,100),
               make_density=0,
             select_level=.5,
              xlabel='$\delta_{R}$')
fig.savefig('{}/stack_plots/{}_{}_{}_drcut.pdf'.format(output_dir, feature, era, reg))

In [ ]:
background_df = df[df.type.str.contains('bck')]
_bhist = make_sys_hist(background_df, feature, reg)

In [ ]:
dy_sel = (df.category.str.contains('DY')) & (df.category != 'DYLL')
dy_types = df[dy_sel].category.unique()

In [ ]:
dy_types

In [ ]:
feature, reg = "DiLepMass", "CR10"
for name in df.name.unique():
    ndf = df[df.name==name]
    _bhist = make_sys_hist(ndf, feature, reg)
    print(name)
    print(_bhist.sys_summary())

In [ ]:
reg = 'CR10'
feature = 'DiLepMass'
fig, ax = plt.subplots(figsize=(12,12))
draw_stackplot(ax, df, feature, reg,era, lumi, draw_bck_sys=1)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))


In [ ]:
for reg in regions:
    fig, ax = plt.subplots(figsize=(12,12))
    draw_stackplot(ax, df, feature, reg,era, lumi)
    print(reg)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, era, reg))


In [ ]:
##
## needs to be run after abcd fits 
##

In [ ]:
file = '{}/fits/limit_setting/abcd_dict_{}.pkl'.format(output_dir, era)
import pickle
import os
import sys
if not os.path.exists(file): sys.exit()

In [ ]:
with open(file, 'rb') as f:
    abcd_fit_dict = pickle.load(f)

In [ ]:
from src.plotting_tools.make_abcd_hist import make_abcd_hist

In [ ]:
for reg, abdc_arrays in abcd_fit_dict.items():
    fig, ax = plt.subplots(figsize=(12,12))
    draw_stackplot(ax, df, feature, reg,era, lumi,draw_bck_sys=True,error_scale=0)
    #abcd hist
    abcd_hist = make_abcd_hist(abdc_arrays)
    abcd_hist.make_density_hist().draw(ax, label='ABCD Pred.', color='green')
    ax.legend(ncol=2)
    fig.savefig('{}/stack_plots/{}_{}_{}_abcd.pdf'.format(output_dir, feature, era, reg))


In [ ]:
##
## needs to be run after limts 
##

In [ ]:
limit_df = pd.read_csv('{}/combine_data/{}_count_limits.csv'.format(outdir, era))

In [ ]:
tldf = limit_df[limit_df.dbs==0.1]
reg1 = tldf[tldf.nJets=='SR1']
reg1 = reg1.sort_values('mass')
count = reg1['50.0']
mass = reg1['mass']

In [ ]:
def interpolate_limit(nJets, mass):
    tldf = limit_df[limit_df.dbs==0.1]
    reg1 = tldf[tldf.nJets==nJets]
    reg1 = reg1.sort_values('mass')
    counts = reg1['50.0']
    masses = reg1['mass']
    return np.interp(mass, masses, counts)

In [ ]:
sdf = df[df.type=='sig']

In [ ]:
#reweight signals to the expected limits
Weight_columns = df.filter(regex='^Weight').columns
for mass in sdf['mass'].unique():
    for dbs in sdf['dbs'].unique():
        for reg in ['SR1', 'SR2']:
            count_limit = interpolate_limit(reg, mass)
            reg_sel = df.filter(regex='{}'.format(reg)).sum(axis=1) > 0
            sample_select = (df.dbs==dbs) & (df.mass==mass)
            nEvents = df[sample_select & (df[reg+'_jet_nom_muon_corrected_pt_ele_pt']==1)].Weight.sum()
            reweight = count_limit/nEvents
            df.loc[sample_select & reg_sel, Weight_columns] *= reweight


In [ ]:
feature = 'DiLepMass'
for reg, abdc_arrays in abcd_fit_dict.items():
    fig, ax = plt.subplots(figsize=(12,12))
    draw_stackplot(ax, df, feature, reg,era, lumi,draw_bck_sys=True,error_scale=0)
    #abcd hist
    abcd_hist = make_abcd_hist(abdc_arrays)
    abcd_hist.make_density_hist().draw(ax, label='ABCD Pred.', color='green')
    ax.legend(ncol=2)
    fig.savefig('{}/stack_plots/{}_{}_{}_abcd_limits.pdf'.format(output_dir, feature, era, reg))


In [ ]:
feature = 'DiLepMass'
bin_edges=np.linspace(110, 200, (200-110+1))
for reg, abdc_arrays in abcd_fit_dict.items():
    fig, ax = plt.subplots(figsize=(12,12))
    draw_stackplot(ax, df, feature, reg,era, lumi,draw_bck_sys=True,error_scale=0, yscale='linear')
    #abcd hist
    abcd_hist = make_abcd_hist(abdc_arrays)
    abcd_hist.make_density_hist().draw(ax, label='ABCD Pred.', color='green')
    ax.legend(ncol=2)
    ax.set_xlim(110,200)
    ax.set_ylim(0,150)
    fig.savefig('{}/stack_plots/{}_{}_{}_abcd_limits_200_max.pdf'.format(output_dir, feature, era, reg))


In [ ]:
##
## jes jer variaitons
##
from src.assets.regions import region_and_label_AN
def sum_region(reg):
    return "{:.1f}".format(df[(df.mass==200) &
        (df[reg]==1)].Weight.sum())

In [ ]:
reg_list = []
for reg, item in region_and_label_AN.items():
    if 'SR' not in reg: continue

    reg_dict = {
        'era': era,
        'reg': item, 
        'nominal': sum_region(reg+'_jet_nom_muon_corrected_pt_ele_pt'),
        'JER Up': sum_region(reg+'_jet_jerUp_muon_corrected_pt_ele_pt'),
        'JER Down': sum_region(reg+'_jet_jerDown_muon_corrected_pt_ele_pt'),
        'JES Up': sum_region(reg+'_jet_jesTotalUp_muon_corrected_pt_ele_pt'),
        'JES Down': sum_region(reg+'_jet_jesTotalDown_muon_corrected_pt_ele_pt'),
    }
    reg_list.append(reg_dict)

In [ ]:
print(pd.DataFrame(reg_list).to_latex())

In [ ]:
from src.plotting_tools.utils import ratio_plot_template

In [ ]:
bck_df = df[df.type.str.contains('bck')]

In [ ]:
relmet_bins = Bins(np.linspace(0, 1,10))
HTLT_bins = Bins(np.linspace(-800,800, 10))

In [ ]:
def make_bff_hist(column, region, var, bins): 
    tdf = bck_df[bck_df['{}_{}'.format(region, var)]>-1]
    return make_hist(tdf['{}_{}'.format(column, var)],
                       bin_edges=bins.bin_edges, weights=tdf.Weight, std=1)

In [ ]:
def make_bff_plot(column, region, bins, xlabel):
    nom, std = make_bff_hist(column, region, 'jet_nom_muon_corrected_pt_ele_pt', bins)
    jerUp, _ = make_bff_hist(column, region, 'jet_jerUp_muon_corrected_pt_ele_pt', bins)
    jerDown, _ = make_bff_hist(column, region, 'jet_jerDown_muon_corrected_pt_ele_pt', bins)
    jesUp, _ = make_bff_hist(column, region, 'jet_jesTotalUp_muon_corrected_pt_ele_pt', bins)
    jesDown, _ = make_bff_hist(column, region, 'jet_jesTotalDown_muon_corrected_pt_ele_pt', bins)
    jer = sorted([jerUp, jerDown], key=lambda x: x.sum())
    jes = sorted([jesUp, jesDown], key=lambda x: x.sum())
    
    down = (((jer[0]-nom)**2 + (jes[0]-nom)**2)**.5)*((jer[0]-nom) + (jes[0]-nom))/abs(((jer[0]-nom) + (jes[0]-nom)))  +nom
    up = (((jer[1]-nom)**2 + (jes[1]-nom)**2)**.5)*((jer[1]-nom) + (jes[1]-nom))/abs(((jer[1]-nom) + (jes[1]-nom)))+nom
    
    fig, ax, rax = ratio_plot_template(figsize=(12,10))
    ax.errorbar(bins.calc_bin_centers(), nom, yerr=std, color='black', label='nom', ds='steps-mid')
    ax.plot(bins.calc_bin_centers(), down, color='red', label='down', ds='steps-mid')
    ax.plot(bins.calc_bin_centers(), up, color='green', label='up', ds='steps-mid')
    
    rax.plot(bins.calc_bin_centers(), nom/nom, color='black', label='nom', ds='steps-mid')
    rax.plot(bins.calc_bin_centers(), down/nom, color='red', label='down', ds='steps-mid')
    rax.plot(bins.calc_bin_centers(), up/nom, color='green', label='up', ds='steps-mid')
    rax.set_ylim(.75,1.25)
    cms_format_fig(era, ax)
    
    ax.set_yscale('log')
    ax.set_ylabel('counts')
    rax.set_ylabel("ratio")
    rax.set_xlabel(xlabel)
    ax.legend()
    return fig

In [ ]:
column, region = 'RelMET', 'SR1'
fig = make_bff_plot(column, region, relmet_bins, RelMET)
fig.savefig('{}/BFF_sys_plots/{}_{}.pdf'.format(output_dir, era, column,))

In [ ]:
column, region = 'HTLT', 'SR1'
fig = make_bff_plot(column, region, HTLT_bins, RelMET)
fig.savefig('{}/BFF_sys_plots/{}_{}.pdf'.format(output_dir, era, column, region))